## This script contains 

This script contains the following:  
1. Importing your libraries and data  
2. Subsetting, wrangling, and cleaning time-series data  
3. Time series analysis: decomposition  
4. Testing for stationarity  
5. Stationarizing the Federal Reserve Economic Data

## 1. Import Libraries and Data

In [1]:
!pip install quandl
!pip install nasdaq-data-link
#import Nasdaqdatalink
import quandl

import sys
print(sys.executable)
import sys
!{sys.executable} -m pip install nasdaq-data-link
import sys
!{sys.executable} -m pip install nasdaq-data-link --upgrade
!{sys.executable} -m pip install quandl

C:\Users\supri\anaconda3\python.exe


In [2]:
import quandl
import nasdaqdatalink
#import nasdaq_data_link
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.api as sm # Using .api imports the public access version of statsmodels, which is a library that handles 
# statistical models.
import os
import warnings # This is a library that handles warnings.


warnings.filterwarnings("ignore") # Disable deprecation warnings that could indicate, for instance, a suspended library or 
# feature. These are more relevant to developers and very seldom to analysts.
import gc
plt.style.use('fivethirtyeight') # This is a styling option for how your plots will appear. More examples here:
# https://matplotlib.org/3.2.1/tutorials/introductory/customizing.html
# https://matplotlib.org/3.1.0/gallery/style_sheets/fivethirtyeight.html

In [3]:
# Configure API key 

#quandl.ApiConfig.api_key = 'PYmzEPFymWTHg-VVGksL'

# Importing Federal Reserve Economic Data  variable.
#data = quandl.get('OPEC/ORB')

In [4]:
#Nasdaqdatalink.ApiConfig.api_key = 'PYmzEPFymWTHg-VVGksL'
#df = Nasdaqdatalink.get_table(dataset='WIKI/APPL', pagiantion = True, api_key = 'PYmzEPFymWTHg-VVGksL')

In [5]:
# Set API key
#nasdaq_data_link.ApiConfig.api_key = 'PYmzEPFymWTHg-VVGksL'

# Fetch data
#df = nasdaq_data_link.get(dataset='WIKI/AAPL', paginate=True)

In [6]:
# Set API key
quandl.ApiConfig.api_key= 'g5rsaJW2evgvB4JYWywC'

# Fetch data
#df = quandl.get(dataset='WIKI/AAPL', paginate=True)
#df = quandl.get("FRED/GDP")
#data = quandl.get('FRED/NROU')

In [7]:
# Import Data
# Create path
path = r'C:\Users\supri\Desktop\Supriya\DataAnalysis\Project\Data Immersion\Achievement 1-2\Boat_Sales'
# Import the merged data set
df = pd.read_csv(os.path.join(path, '02 Data', 'Prepared data', 'FRED-NROU.csv'), index_col = 0)

In [8]:
df.head(5)

,VALUE
DATE,
2016-10-01,4.8
2016-07-01,4.8
2016-04-01,4.8
2016-01-01,4.9
2015-10-01,4.9


In [9]:
df.shape

(272, 1)

In [10]:
df.columns

Index(['VALUE'], dtype='object')

In [11]:
type(df)

pandas.core.frame.DataFrame

In [12]:
# Plot the data using matplotlib.


df['DATE'] = pd.to_datetime(df['DATE'])

plt.figure(figsize=(15,5), dpi=100)
plt.plot(df['DATE'], df['VALUE'])
plt.show()


KeyError: 'DATE'

### 2. Subsetting, wrangling, and cleaning time-series data

In [ ]:
data_2 = df.drop(columns=['SMA', 'Month', 'Year', 'Date_new'], errors='ignore').reset_index()

In [ ]:
data_2.head()

In [ ]:
data_2 = data_2.drop(columns=['index'])

In [ ]:
data_sub = data_2.loc[(data_2['DATE'] >= '1980-01-01') & (data_2['DATE'] < '2020-06-01')]

In [ ]:
data_sub.shape

In [ ]:
data_sub.head()

In [ ]:
# Set the "Date" column as the index

from datetime import datetime

data_sub['datetime'] = pd.to_datetime(data_sub['DATE']) # Create a datetime column from "Date.""
data_sub = data_sub.set_index('datetime') # Set the datetime as the index of the dataframe.
data_sub.drop(['DATE'], axis=1, inplace=True) # Drop the "Date" column.
data_sub.head()

In [ ]:
# Plot the new data set

plt.figure(figsize=(15,5), dpi=100)
plt.plot(data_sub)

In [ ]:
# Check for missing values (you shouldn't have any)

data_sub.isnull().sum() 

In [ ]:
# Check for duplicates

dups = data_sub.duplicated()
dups.sum()

In [ ]:
for col in data_sub.columns:
    num_duplicates = data_sub[col].duplicated().sum()
    print(f"Column '{col}' has {num_duplicates} duplicate values.")

##### Time series data is indexed by time (DATE), so uniqueness usually depends on the time points.It’s common to have the same VALUE at different time points so not removing the duplicate vlaues.

### 3. Time-series analysis: decomposition

In [ ]:
# Decompose the time series using an additive model

decomposition = sm.tsa.seasonal_decompose(data_sub, model='additive')

In [ ]:
from pylab import rcParams # This will define a fixed size for all special charts.

rcParams['figure.figsize'] = 18, 7

In [ ]:
# Plot the separate components

decomposition.plot()
plt.show()

### 4. Testing for stationarity

In [ ]:
# The adfuller() function will import from the model from statsmodels for the test; however, running it will only return 
# an array of numbers. This is why you need to also define a function that prints the correct output from that array.

from statsmodels.tsa.stattools import adfuller # Import the adfuller() function

def dickey_fuller(timeseries): # Define the function
    # Perform the Dickey-Fuller test:
    print ('Dickey-Fuller Stationarity test:')
    test = adfuller(timeseries, autolag='AIC')
    result = pd.Series(test[0:4], index=['Test Statistic','p-value','Number of Lags Used','Number of Observations Used'])
    for key,value in test[4].items():
       result['Critical Value (%s)'%key] = value
    print (result)

# Apply the test using the function on the time series
dickey_fuller(data_sub['VALUE'])

In [ ]:
# Check out a plot of autocorrelations

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf # Here, you import the autocorrelation and partial correlation plots

plot_acf(data_sub)
plt.show()

### 5. Stationarizing the Federal Reserve Economic Data

In [ ]:
data_diff = data_sub - data_sub.shift(1) # The df.shift(1) function turns the observation to t-1, making the whole thing t - (t -1)

In [ ]:
data_diff.dropna(inplace = True) # Here, you remove the missing values that came about as a result of the differencing. 
# You need to remove these or you won't be able to run the Dickey-Fuller test.

In [ ]:
data_diff.head()

In [ ]:
data_diff.columns

In [ ]:
# Check out what the differencing did to the time-series curve

plt.figure(figsize=(15,5), dpi=100)
plt.plot(data_diff)

In [ ]:
dickey_fuller(data_diff)

In [ ]:
plot_acf(data_diff)
plt.show()